# Trabalhando com a api da Open AI

Maior parte dos comentários serão em inglês para adaptação a língua.

É interessante testar os prompts e os parâmetros no playground:
https://platform.openai.com/playground/prompts?models=gpt-4.1

##### Instalando a biblioteca
!pip install openai

!pip install dotenv

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() # Carrega as variáveis do arquivo .env
openai_api_key = os.getenv("OPENAI_API_KEY") # Acessa a variável como se fosse do sistema

In [2]:
from openai import OpenAI

In [3]:
# Biblioteca para exibir resultados em markdown
from IPython.display import Markdown, display

In [4]:
# Define the URL of an image we will use later
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"

In [5]:
MODEL = "gpt-4o"

In [6]:
# Connect to the api key
client = OpenAI(api_key = openai_api_key)

## Texto

In [7]:
# Define the system prompt
system_prompt = "You are Vegeta before Dragon Ball Super"

# Define the user prompt
user_prompt = "have you defeated Goku sometime?"

In [8]:
# Generating text with OpenAI
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_prompt}
    ]
)

In [9]:
# Displaying output
#print(response.choices[0].message.content)
display(Markdown(response.choices[0].message.content))

As Vegeta, I've faced Goku in battle several times throughout our history. The first major battle was when he first came to Earth. While I was initially stronger and nearly defeated him, things didn't go as planned, largely due to the intervention of his friends, as well as Goku’s incredible ability to grow stronger during a fight.

Despite my early advantage, Goku and his allies managed to turn the tide. In subsequent battles, we continued to push each other to new limits. While I haven't decisively defeated Goku, our rivalry drives us both to become stronger. Each battle only serves to heighten our competitive spirit and mutual respect.

## Text Generation with Parameters

Geralmente é recomendável alterar "temperature" ou "top_t", mas não ambos

In [10]:
# Define the system and user prompt
system_prompt = "You are Vegeta before Dragon Ball Super"
user_prompt = "have you defeated Goku sometime?"

# Generating text with OpenAI
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_prompt}
    ],
    temperature=1.1, # Precisão vs Criatividade. Entre 0 e 2.0. Padrão 1.0
    top_p=0.8, # Porcentagem de gama de palavras consideradas para geração da proxima palavra. Ex: 0.3 = 30%
    presence_penalty=0.5, # Penaliza repetição de ideias ou temas. Entre -2.0 e 2.0. Padrão 0
    frequency_penalty=0 # Penaliza repetição de palavras. Entre -2.0 e 2.0. Padrão 0
)

# Displaying output
display(Markdown(response.choices[0].message.content))

I came close to defeating Kakarot during our first battle on Earth. I was definitely stronger than him at that time, and I had the upper hand for most of the fight. However, his stubbornness and the intervention of his friends turned the tide against me. Since then, we've both grown stronger and have had our share of intense battles, but outright defeating him has been elusive. Still, I'm always striving to surpass him.

Há outros parâmetros a usar. Consultar: https://platform.openai.com/docs/api-reference/chat/create

## Imagens

#### Imagem por url

In [22]:
# Define the user content
user_content = [
    {"type": "text",
     "text": "Describe the image"},
    {"type": "image_url",
     "image_url": {
         "url": url,
         "detail": "high" # Opcional. Se a llm deve analisar a imagem em grandes detalhes ou pequenos detalhes.
     }}
]

In [13]:
# Use the chat.completions
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": user_content}
    ]
)
display(Markdown(response.choices[0].message.content))

The image depicts a serene landscape featuring a wooden boardwalk winding through a lush green meadow. Lively grass surrounds the walkway, and the scene is set under a bright blue sky with scattered white clouds. In the background, there are trees and shrubs, adding depth to the peaceful, natural setting. The lighting suggests it is a bright, sunny day.

#### Imagem em base64

In [14]:
import base64

In [16]:
file_name = "4.open_ai/Thumbnail python FV1.jpg" # pasta e arquivo dentro do projeto
file_path = os.path.join(os.getcwd(), file_name) # os.getcwd() pega o caminho atual do projeto

In [18]:
# Read image and convert to base64
with open(file_path, "rb") as image_file: # rb significa "read binary"
    image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

In [23]:
# Define user content
user_content = [
    {"type": "text",
     "text": "This is my thumbnail for Data Analysis course. Be brutal, mean and provide sarcastic suggestions"},
    {"type": "image_url",
     "image_url": {
         "url": f"data:image/jpeg;base64,{image_base64}", #significado = data que é imagem que é jpeg mas em base64. É uma sintaxe padrão.
         "detail": "high"
     }}
]

In [20]:
# Call the OpenAI API
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": user_content}
    ]
)
display(Markdown(response.choices[0].message.content))

Ah, the classic "snake charmer" approach to data analysis. Bold choice! The recorder really screams "professional muso," doesn't it? Very avant-garde. Your background of binary numbers really drives home that complex, matrix-y vibe. Just be careful; you might hypnotize the entire internet with this masterpiece. Don't you just love how the colors clash ever so gracefully? It's like a visual symphony! Maybe add a few more random elements, like a cat on a unicycle, just for that extra sprinkle of chaos. Perfect for expressing the wild, unpredictable world of data. Bravo!

#### TESTES PESSOAIS

In [33]:
def generate(file_path, query, model, client):
    
    with open(file_path, "rb") as image_file: # rb significa "read binary"
        image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
    user_content = [
        {"type": "text",
         "text": query},
        {"type": "image_url",
         "image_url": {
             "url": f"data:image/jpeg;base64,{image_base64}",
             "detail": "high"
         }}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": user_content}
        ]
    )
    return response.choices[0].message.content

In [34]:
file_name = "4.open_ai/Orizonte da Praia.jpg"
file_path = os.path.join(os.getcwd(), file_name)
query = "Desenhei esse desenho copiando uma imagem. Descreva a imagem e depois faça uma critica maldosa com sugestões sarcasticas."

In [35]:
resposta = generate(file_path, query, MODEL, client)
display(Markdown(resposta))

A imagem mostra um desenho em lápis de dois coqueiros inclinados à beira de uma praia. Ao fundo, há o mar e algumas formações montanhosas. No canto inferior, uma cerca rústica e um pequeno barco estão presentes. Algumas aves estão voando no céu.

Agora, a crítica maldosa:

Parabéns por capturar a essência de uma "obra-prima em potencial"! É admirável como os coqueiros parecem ter um bad hair day tropical, e a perspectiva? Revolucionária! Quem precisa de proporção quando se pode desafiar as leis da física e da botânica ao mesmo tempo? A sugestão do barco é um toque de mestre – empresários do turismo definitivamente vão querer se inspirar nesse design. E o que dizer das montanhas? Esta é uma nova abordagem de topografia sem compromisso com a realidade.

Para sua próxima criação, talvez considere desenhar a Mona Lisa em um cenário lunar. Afinal, a realidade nunca deve ser um obstáculo à sua criatividade singular.